In [111]:
import re
import collections
import os
import time

In [112]:
def tokenize(f):
    #Returns [(term, pos), (term, pos) ...]
    terms = []
    with open("collection/" + f) as file:
        line = file.readline()
        while line:
            # Regex to match only strings and spaces 
            line = re.sub(r'[^A-Za-z\s]+', '', line)
            for i, word in enumerate(line.split()):
                terms.append((word.lower(), file.tell()+i))
            line = file.readline() 
    return terms

In [113]:
def createPostingList():
    postingList = collections.defaultdict(list)
    curDocID = 0

    for f in os.listdir("collection"):
        terms = tokenize(f)

        # create map {word:[pos1, pos2, pos3]}
        wordToPos = collections.defaultdict(list)
        for word, pos in terms:
            wordToPos[word].append(pos)

        # append to posting list {term : [(docID1, [pos1, pos2, pos3, pos4])]}
        for term, arr in wordToPos.items():
            postingList[term].append((curDocID, wordToPos[term]))

        # For every file update id 
        curDocID += 1
    
    return postingList
    

In [114]:
startTime = time.time()
result = createPostingList()
endTime = time.time()

In [115]:
print(f"Index built in {endTime - startTime} seconds.")

Index built in 0.7736330032348633 seconds.


In [120]:
"""
cat = [(6, [1174]), (32, [972]), (74, [1911, 3229]), (162, [54, 199]), (195, [2837]), (249, [1738])]
dog = [(32, [409]), (41, [1541])]

merged = [32]
"""

[(6, [1174]), (32, [972]), (74, [1911, 3229]), (162, [54, 199]), (195, [2837]), (249, [1738])]


In [130]:
def mergeTwo(A, B):
    if not B:
        return [i for i, arr in A]

    a = b = 0
    res = []
    while a < len(A) and b < len(B):
        if A[a][0] > B[b][0]:
            b += 1
        elif A[a][0] < B[b][0]:
            a += 1
        else:
            res.append(A[a][0])
            a += 1
            b += 1
    return res

In [ ]:
the = result["the"]
cat = result["cat"]

print(mergeTwo(the, cat))

In [157]:
query = "Show me a cat"
fixedQuery = re.sub(r'[^A-Za-z\s]+', '', query)
queryTerms = fixedQuery.lower().split()

postingLists = [result[q] for q in queryTerms]

justDocs = []
for word in queryTerms:
    r = result[word]
    cur = []

print(postingLists)

while len(postingLists) > 1:
    temp = []
    for i in range(0, len(postingLists), 2):
        temp.append(mergeTwo(postingLists[i], postingLists[i+1]))
    postingLists = temp


[[(7, [3554]), (8, [10016]), (32, [973]), (38, [1551]), (48, [1408]), (52, [840]), (53, [906]), (59, [9792]), (92, [1383]), (104, [7221]), (109, [2156]), (115, [2358]), (119, [207]), (122, [17491]), (131, [4439]), (133, [1945]), (137, [2026]), (154, [2483]), (206, [2244]), (209, [30238]), (224, [1476]), (232, [463, 2162]), (235, [3792]), (237, [3005]), (241, [1672]), (242, [1401]), (259, [2004]), (266, [2459]), (292, [5566]), (319, [1028]), (344, [4247]), (363, [429]), (400, [1308])], [(2, [5756]), (8, [9177, 9541, 9606, 10633, 11669]), (14, [2638]), (21, [3001]), (25, [2793]), (35, [1811]), (39, [1727, 1798, 1875, 2368]), (41, [350]), (71, [4772]), (78, [1600, 1656]), (81, [2034]), (90, [2111]), (100, [224]), (108, [355]), (109, [2362]), (116, [1572]), (120, [210, 219, 352]), (128, [1118]), (130, [285]), (158, [2748, 4729]), (159, [2220]), (163, [4063, 4126]), (164, [1185, 1193, 1197]), (175, [141]), (177, [2373, 2591, 10210, 14765]), (179, [620]), (181, [4503, 4720]), (184, [829]), (

TypeError: 'int' object is not subscriptable